In [1]:
__author__ = 'Pratyush'
import json
import numpy as np
import pandas as pd
from openpyxl import Workbook

import requests
import json
import os
import modules.config as config
from time import sleep
from requests.auth import HTTPBasicAuth
from ibm_watson import DiscoveryV1
from pprint import pprint
import modules.discovery_helper as dh_help
import modules.common_helper as cm_help
import sys
import pandas as pd
import modules.config as config
import math
import re
import hashlib
import unicodedata
from bs4 import BeautifulSoup 
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from IPython.core.display import display, HTML

all_params = cm_help.get_params()

def discovery_init(COLLECTION_NAME=config.DISCOVERY_COLLECTION_NAME,default=all_params['discovery']['default']):
    response = {}
    
    discovery_param = all_params['discovery']
    authenticator = IAMAuthenticator(apikey=discovery_param[default]['apikey'])
    endurl = "https://gateway.watsonplatform.net/discovery/api"
    discovery = DiscoveryV1(
        version=discovery_param[default]['version'],
        authenticator = authenticator)
    iam_apikey=discovery_param[default]['apikey'],
    url=discovery_param[default]['url']    
    if default =="r3":
        discovery.set_service_url(discovery_param[default]['url'])

    environments = discovery.list_environments().get_result()
    print("Discovery instance loaded "+default)
    js = json.dumps(environments)
    jres = json.loads(js)
    env_id = jres['environments'][1]['environment_id']
    collections = discovery.list_collections(env_id).get_result()
    cols = json.dumps(collections, indent=2)
    colres = json.loads(cols)

    for item in colres['collections']:
        if item['name']== COLLECTION_NAME:
            print('COLLECTION ID:', item['collection_id'], 'COLLECTION NAME:', item['name'])
            col_id = item['collection_id']
    response['env_id'] = env_id
    response['col_id'] = col_id
    response['discovery'] = discovery
    return response

discovery_config = discovery_init(COLLECTION_NAME='Naviga1',default="r3")

Discovery instance loaded r3
COLLECTION ID: ec586f27-f848-40b7-99af-dab80a43b248 COLLECTION NAME: Naviga1


In [3]:
offset = 0
count = 100 # 100 docs 
reqparams = {'offset':offset,'count':count}
reqparams['q'] = 'Last Dance'
discovery_data = dh_help.getDiscoveryData(discovery_config,reqparams)

offset in query = 0


In [4]:
len(discovery_data["results"])

100

In [5]:
len(discovery_data["passages"])

50

In [6]:
discovery_data["passages"]

[{'document_id': '03a09a1c-cd3d-4561-9086-6933b7a19386',
  'passage_score': 38.995676416151014,
  'passage_text': "AHS sal speech: If it's the last dance, dance backward, Class of 2020",
  'start_offset': 0,
  'end_offset': 69,
  'field': 'title'},
 {'document_id': '50602105-0b9c-42f9-9bd7-01641652a2ba',
  'passage_score': 35.62804748510827,
  'passage_text': "Tune In Tonight: Little League reruns and more 'Last Dance'",
  'start_offset': 0,
  'end_offset': 59,
  'field': 'title'},
 {'document_id': '788ed57f-834f-4e5b-949f-7ccddf1d76b5',
  'passage_score': 35.62804748510827,
  'passage_text': "Tune In Tonight: Little League reruns and more 'Last Dance'",
  'start_offset': 0,
  'end_offset': 59,
  'field': 'title'},
 {'document_id': '03a09a1c-cd3d-4561-9086-6933b7a19386',
  'passage_score': 29.287775093359205,
  'passage_text': "Sidney Irwin is salutatorian of Abilene High School. 'If it's the last dance, dance backwards.'. When I first heard this quote, it sounded to me like one of tho

In [110]:
for i in range(100):
    if(discovery_data["results"][i]["id"] == discovery_data["passages"][0]["document_id"]):
        source = discovery_data["results"][i]["html"]

In [111]:
ps_text = discovery_data["passages"][3]["passage_text"]

In [112]:
ps_text

"Sidney Irwin is salutatorian of Abilene High School. 'If it's the last dance, dance backwards.'. When I first heard this quote, it sounded to me like one of those vague little sayings people write to sound poetic."

In [113]:
source

"\n<p><person>Sidney Irwin</person> is salutatorian of <org>Abilene High School</org>.</p>\n<p>'If it's the last dance, dance backwards.'</p>\n\n<p>When I first heard this quote, it sounded to me like one of those vague little sayings people write to sound poetic. But when I really thought about it, what it's trying to tell you, it's kind of beautifully true.</p>\n\n<p>In the 13 years that we have been going to school, we have gone through a lot of changes in our lives. We've made friends, lost friends, moved schools, joined clubs, aced projects and failed papers and learned more about ourselves along the way.</p>\n\n<p>We've felt joy and terror, sadness and hope, we've learned to love and learned to lose and sometimes lost ourselves in the in between.</p>\n\n<p>We've grown up.</p>\n\n<p>And every time we have these changes, the big ones or the small ones, we look around at all our friends and our first thoughts are not of the future, of what's up ahead, because we don't know what's co

In [180]:
offset = 0
count = 100
reqparams = {'offset':offset,'count':count}
reqparams['q'] = 'Inflation'
discovery_data_subCode = dh_help.getDiscoveryData(discovery_config,reqparams)

doc_id=[]
passage=[]
for i in range(len(discovery_data_subCode["passages"])):
    doc_id.append(discovery_data_subCode["passages"][i]["document_id"])
    passage.append(discovery_data_subCode["passages"][i]["passage_text"])

HTM=[]
for j in range(10):
    reqparams = {'offset':offset,'count':count}
    reqparams['filters']='id::'+doc_id[j]
    discovery_data2 = dh_help.getDiscoveryData(discovery_config,reqparams)
    HTM.append(discovery_data2["results"][0]["html"])

offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0
offset in query = 0


In [3]:
len(doc_id)

50

In [181]:
# len(passage)
passage

['PENN STATE/ACY ALTERNATIVE INFLATION INDEX REVEALS DECLINE IN INFLATION RATE [U.S. Fed News]',
 'Inflation declines 5.35% in May',
 'Inflation declines to 5.35pc in May',
 'Inflation rate sustains downtrend: Diokno',
 'WING INFLATABLES Wins $250,000 Federal Contract',
 "The SBP tweeted: 'A chart from our Inflation Monitor has generated erroneous headlines. It compares Pakistan's inflation to a few developed and developing countries. While Pakistan did have a relatively elevated inflation rate so far in FY20, it ...",
 'Inflation slides to 2.1% in April.(Business News)',
 "Does Pakistan have the world's highest inflation? No, says SBP",
 'May 2020 inflation rate clocked in at 0.07 percent: BPS',
 'Good news: Inflation down, recovery up.(Opinions and Editorials)',
 'Inflation fears rise in households: RBI poll [Mint (New Delhi)]',
 'Camano Fire crews rescue man after inflatable canoe sinks in Port Susan Bay',
 'Pandemic fallout to keep Philippine inflation in check, BSP says [Philippin

In [224]:
h=HTM[0]
p=passage[0]

In [225]:
p

'PENN STATE/ACY ALTERNATIVE INFLATION INDEX REVEALS DECLINE IN INFLATION RATE [U.S. Fed News]'

In [226]:
display(HTML(h))

In [222]:
lists1 = p.split('. ')
flag=0
for j in range(len(lists1)):
    lists2 = lists1[j].split()
    for g in range(len(lists2)):
        lists2[g] = re.sub('\W+','', lists2[g])
    for i in range(len(lists2)):
        batRegex = re.compile("\\b"+lists2[i]+"\\b")
        places = [(m.start(0), m.end(0)) for m in re.finditer(batRegex, h)]
        if len(places)==0:
            batRegex = re.compile("\\b"+lists2[i].lower()+"\\b")
            places = [(m.start(0), m.end(0)) for m in re.finditer(batRegex, h)]
        if (len(places)!=0):
            if i == len(lists2)-1:
                batRegex2 = re.compile("\\b"+lists2[i]+"\\b")
                places2 = [(m.start(0), m.end(0)) for m in re.finditer(batRegex2, h)]
                if len(places2)!=0:
                    for z in range(len(places2)):
                        if (places2[z][0]-final_highlight>0 and places2[z][0]-final_highlight<50):
                            part = h[places2[z][0]:places2[z][1]+1]
#                             h = h.replace(part,"<mark><b>"+part+"</b></mark>".format(part),1)
                            h = h[:places2[z][0]] + "<mark><b>"+part+"</b></mark>" + h[places2[z][1]+1:]
                            break
                
                
                
            for k in lists2[i+1:]:
                batRegex1 = re.compile("\\b"+k+"\\b")
                places1 = [(m.start(0), m.end(0)) for m in re.finditer(batRegex1, h)]
                if len(places1)!=0:
                    for x in range(len(places)):
                        for y in range(len(places1)):
                            if(places1[y][0]-places[x][1]<50 and places1[y][0]-places[x][1]>0):
                                part = h[places[x][0]:places[x][1]+1]
#                                 h = h.replace(part,"<mark><b>"+part+"</b></mark>".format(part),1)
                                h = h[:places[x][0]] + "<mark><b>"+part+"</b></mark>" + h[places[x][1]+1:]
                                if i == len(lists2)-2:
                                    final_highlight = places[x][1]
                                flag=1
                                break
                        if flag==1:
                            break
                if flag==1:
                    break
        flag=0
                
                            
                        
                        

In [223]:
display(HTML(h))

In [43]:
lists1 = p.split('. ')
for j in range(len(lists1)):
    lists2 = lists1[j].split()
    for i in range(len(lists2)):
        if(len(lists2[i])>4):
            a = h.find(lists2[i])
            if(a==-1):
                a = h.find(lists2[i].lower())
            if(a!=-1):
                part = h[a:a+len(lists2[i])]
                for k in lists2 :                    
                    if ((abs(h.find(k)-a)<50) and (k!=lists2[i])):
                        h = h.replace(part,"<mark><b>"+part+"</b></mark>".format(part))
                        print("done")
                        break

53
done


In [ ]:
batRegex = re.compile("\\binflation\\b")
# lists = re.search(batRegex,h)
# indicesTuple=[(mObj.start(1),mObj.end(1)-1) for mObj in re.finditer(batRegex,h)]
listsss= re.finditer(batRegex, h)
places = [(m.start(0), m.end(0)) for m in re.finditer(batRegex, h)]
places

In [107]:
display(HTML(h))

In [412]:
for i in range(len(lists1)):
    
    lists2 = lists1[i].split()

    for j in range(len(lists2)):
        lists2[j] = re.sub('\W+','', lists2[j])
    
#     first = lists2[0]
#     last = lists2[-1]
#     if(len(last)<3):
#         if(len(lists2)>1):
#             last=lists2[-2]
    first = lists2[0]
    last = lists2[-1]

    z = len(lists2)
    i=1
    while(z>0):
        last=lists2[-i]
        if(len(last)<3):
            i+=1
            z-=1
        else:
            break
    
    a = h1.find(first)
    b = h1.find(last)
    part1 = h1[a:b+len(last)]
    part2 = part1.split("\n")
    s="\n"

    for i in range(len(part2)):  

        if first and last in part2[i]:

            part2[i] = part2[i].replace(first, "<mark><b>"+ first +"</mark></b>".format(i))
            part2[i] = part2[i].replace(last, "<mark><b>"+ last +"</mark></b>".format(i))

    part1=s.join(part2)
    h1 = h1.replace(h1[a:b+len(last)],part1)
    batRegex = re.compile(r'(.</b>(.+?)<mark><b>)')  #(.+?)
    l = re.findall(batRegex,h1)
    for g in range(len(l)):
        l[g] = list(l[g])
    for g in range(len(l)):
        if(len(l[g][1])>1):
            h1 = h1.replace(l[g][0],"></b><mark><b>"+ l[g][1] +"</mark></b><mark><b>")


In [413]:
display(HTML(h1))

In [397]:
first = lists2[0]
last = lists2[-1]

z = len(lists2)
i=1
while(z>0):
    last=lists2[-i]
    if(len(last)<3):
        i+=1
        z-=1
    else:
        break

print(first,last)

While it


In [357]:
stri = "hello Hello"

In [359]:
stri.find("Hello")

6

In [356]:
a = h1.find(first)
b = h1.find(last) #[^a-z]


730 376


In [362]:
h1[376:400]

'itor has generated erron'

In [ ]:
a = h1.find(first)
b = h1.find(last)

part1 = h1[a:b+len(last)]
part2 = part1.split("\n")
s="\n"

for i in range(len(part2)):  

    if first and last in part2[i]:

        part2[i] = part2[i].replace(first, "<mark><b>"+ first +"</mark></b>".format(i))
        part2[i] = part2[i].replace(last, "<mark><b>"+ last +"</mark></b>".format(i))

part1=s.join(part2)
h1 = h1.replace(h1[a:b+len(last)],part1)
batRegex = re.compile(r'(.</b>(.+?)<mark><b>)')  #(.+?)
l = re.findall(batRegex,h1)
for g in range(len(l)):
    l[g] = list(l[g])
for g in range(len(l)):
    if(len(l[g][1])>1):
        h1 = h1.replace(l[g][0],"></b><mark><b>"+ l[g][1] +"</mark></b><mark><b>")

In [352]:
display(HTML(h1))

In [300]:
pprint(h1)

('\n'
 '<p> The <org idsrc="xmltag.org" value="ACORN:0966177357">State Bank of '
 'Pakistan</org> (SBP) said that <location idsrc="xmltag.org" '
 'value="LC/pk">Pakistan\'s</location> inflation for the year 2020 was not the '
 'highest in the world, in a series of tweets on Sunday.</p>\n'
 "<p><mark><b>The</mark></b><mark><b> SBP tweeted: 'A chart from our Inflation "
 'Monitor has generated erroneous </mark></b><mark><b>headlines</mark></b>. It '
 'compares <location idsrc="xmltag.org" value="LC/pk">Pakistan\'s</location> '
 'inflation to a few developed and developing countries. While <location '
 'idsrc="xmltag.org" value="LC/pk">Pakistan</location> did have a relatively '
 'elevated inflation rate so far in FY20, it is not true that it was the '
 "highest in the world.'</p>\n"
 '<p>\'For instance, <location idsrc="xmltag.org" '
 'value="LC/ar">Argentina</location>, <location idsrc="xmltag.org" '
 'value="LC/ir">Iran</location>, <location idsrc="xmltag.org" '
 'value="LC/ng">Nigeria

In [136]:
def finding_fragment(passage,html,empty=0):
    str_=" "
    i=0
    a=passage.split()[empty:]
    frag=a[0]
    while (frag in html):
        i+=1
        frag=frag+" "+a[i]
    str_=str_.join(frag.split()[:-1])
    if len(str_)!=0:
        html=html.replace(str_,"<mark><b>"+ str_ +"</mark></b>",1)
    passage=passage.replace(str_,"")
    return (passage,str_,html)

In [137]:
empty=0

In [138]:
while(len(p.split())>empty):
    p,s,h=finding_fragment(p,h,empty=empty)
    if len(s)==0:
        empty+=1

In [139]:
display(HTML(h))

# With html2text

In [1]:
import html2text

In [91]:
h = html2text.HTML2Text()
h.ignore_links = True
h = h.handle(source)


In [92]:
h

"Sidney Irwin is salutatorian of Abilene High School.\n\n'If it's the last dance, dance backwards.'\n\nWhen I first heard this quote, it sounded to me like one of those vague little\nsayings people write to sound poetic. But when I really thought about it, what\nit's trying to tell you, it's kind of beautifully true.\n\nIn the 13 years that we have been going to school, we have gone through a lot\nof changes in our lives. We've made friends, lost friends, moved schools,\njoined clubs, aced projects and failed papers and learned more about ourselves\nalong the way.\n\nWe've felt joy and terror, sadness and hope, we've learned to love and learned\nto lose and sometimes lost ourselves in the in between.\n\nWe've grown up.\n\nAnd every time we have these changes, the big ones or the small ones, we look\naround at all our friends and our first thoughts are not of the future, of\nwhat's up ahead, because we don't know what's coming.\n\nWe can't predict what's going to happen next. I mean, lo

In [93]:
h = h.replace('\n\n',' ')


In [94]:
h

"Sidney Irwin is salutatorian of Abilene High School. 'If it's the last dance, dance backwards.' When I first heard this quote, it sounded to me like one of those vague little\nsayings people write to sound poetic. But when I really thought about it, what\nit's trying to tell you, it's kind of beautifully true. In the 13 years that we have been going to school, we have gone through a lot\nof changes in our lives. We've made friends, lost friends, moved schools,\njoined clubs, aced projects and failed papers and learned more about ourselves\nalong the way. We've felt joy and terror, sadness and hope, we've learned to love and learned\nto lose and sometimes lost ourselves in the in between. We've grown up. And every time we have these changes, the big ones or the small ones, we look\naround at all our friends and our first thoughts are not of the future, of\nwhat's up ahead, because we don't know what's coming. We can't predict what's going to happen next. I mean, look at right now. Who\

In [95]:
h = h.replace('\n',' ')
h

"Sidney Irwin is salutatorian of Abilene High School. 'If it's the last dance, dance backwards.' When I first heard this quote, it sounded to me like one of those vague little sayings people write to sound poetic. But when I really thought about it, what it's trying to tell you, it's kind of beautifully true. In the 13 years that we have been going to school, we have gone through a lot of changes in our lives. We've made friends, lost friends, moved schools, joined clubs, aced projects and failed papers and learned more about ourselves along the way. We've felt joy and terror, sadness and hope, we've learned to love and learned to lose and sometimes lost ourselves in the in between. We've grown up. And every time we have these changes, the big ones or the small ones, we look around at all our friends and our first thoughts are not of the future, of what's up ahead, because we don't know what's coming. We can't predict what's going to happen next. I mean, look at right now. Who could ha

In [96]:
source = h

In [97]:
ps_text

"Sidney Irwin is salutatorian of Abilene High School. 'If it's the last dance, dance backwards.'. When I first heard this quote, it sounded to me like one of those vague little sayings people write to sound poetic."

In [69]:
lists = ps_text.split(" ")

In [52]:
lists

['Sidney',
 'Irwin',
 'is',
 'salutatorian',
 'of',
 'Abilene',
 'High',
 'School.',
 "'If",
 "it's",
 'the',
 'last',
 'dance,',
 'dance',
 "backwards.'.",
 'When',
 'I',
 'first',
 'heard',
 'this',
 'quote,',
 'it',
 'sounded',
 'to',
 'me',
 'like',
 'one',
 'of',
 'those',
 'vague',
 'little',
 'sayings',
 'people',
 'write',
 'to',
 'sound',
 'poetic.']

In [53]:
first = lists[0]
last = lists[-1]

In [44]:
import os

In [ ]:
# 0 black
# 1 red
# 2 green
# 3 yellow
# 4 blue
# 5 magenta
# 6 cyan
# 7 white
# 9 default

In [50]:
if "Sidney" in source:
    print()
    print(source.replace("Sidney", '\033[43;34m{}\033[m'.format("Sidney")))


Sidney Irwin is salutatorian of Abilene High School. 'If it's the last dance, dance backwards.' When I first heard this quote, it sounded to me like one of those vague little sayings people write to sound poetic. But when I really thought about it, what it's trying to tell you, it's kind of beautifully true. In the 13 years that we have been going to school, we have gone through a lot of changes in our lives. We've made friends, lost friends, moved schools, joined clubs, aced projects and failed papers and learned more about ourselves along the way. We've felt joy and terror, sadness and hope, we've learned to love and learned to lose and sometimes lost ourselves in the in between. We've grown up. And every time we have these changes, the big ones or the small ones, we look around at all our friends and our first thoughts are not of the future, of what's up ahead, because we don't know what's coming. We can't predict what's going to happen next. I mean, look at right now. Who could ha

In [57]:
for i in [first,last]:
    if i in source:
        source = source.replace(i, '\033[43;34m{}\033[m'.format(i))
print(source)

Sidney Irwin is salutatorian of Abilene High School. 'If it's the last dance, dance backwards.' When I first heard this quote, it sounded to me like one of those vague little sayings people write to sound poetic. But when I really thought about it, what it's trying to tell you, it's kind of beautifully true. In the 13 years that we have been going to school, we have gone through a lot of changes in our lives. We've made friends, lost friends, moved schools, joined clubs, aced projects and failed papers and learned more about ourselves along the way. We've felt joy and terror, sadness and hope, we've learned to love and learned to lose and sometimes lost ourselves in the in between. We've grown up. And every time we have these changes, the big ones or the small ones, we look around at all our friends and our first thoughts are not of the future, of what's up ahead, because we don't know what's coming. We can't predict what's going to happen next. I mean, look at right now. Who could hav

In [100]:
for ps_text in source:
    if ps_text in source:
        source = source.replace(ps_text, '\033[43;34m{}\033[m'.format(ps_text))
print(source)

KeyboardInterrupt: 

In [99]:
ps_text

"Sidney Irwin is salutatorian of Abilene High School. 'If it's the last dance, dance backwards.'. When I first heard this quote, it sounded to me like one of those vague little sayings people write to sound poetic."

In [98]:
source

"Sidney Irwin is salutatorian of Abilene High School. 'If it's the last dance, dance backwards.' When I first heard this quote, it sounded to me like one of those vague little sayings people write to sound poetic. But when I really thought about it, what it's trying to tell you, it's kind of beautifully true. In the 13 years that we have been going to school, we have gone through a lot of changes in our lives. We've made friends, lost friends, moved schools, joined clubs, aced projects and failed papers and learned more about ourselves along the way. We've felt joy and terror, sadness and hope, we've learned to love and learned to lose and sometimes lost ourselves in the in between. We've grown up. And every time we have these changes, the big ones or the small ones, we look around at all our friends and our first thoughts are not of the future, of what's up ahead, because we don't know what's coming. We can't predict what's going to happen next. I mean, look at right now. Who could ha